<a href="https://colab.research.google.com/github/Shobanagithub/CODSOFT/blob/main/TASK5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
df=pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [39]:
df.shape

(1986, 31)

In [40]:
df.dtypes

Time        int64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class     float64
dtype: object

In [41]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [52]:
fraud_count=df['Class'].value_counts()
fraud_rate = 100 * fraud_count[1] / df.shape[0]
fraud_rate_series = pd.Series(fraud_rate, name='Rate')
fraud_data = pd.concat([fraud_count, fraud_rate_series], axis=1).reset_index()
fraud_data.columns = ['Class', 'Count', 'Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,1983,0.100705
1,1.0,2,NaN


In [53]:
df_fraud = df[df['Class']==1]
df_not_fraud = df[df['Class']==0]
df_not_fraud_sampled = df_not_fraud.sample(df_fraud.shape[0],replace= False, random_state=101)
df_balanced = pd.concat([df_fraud, df_not_fraud_sampled], axis = 0).sample (frac=1, replace= False, random_state=101).reset_index().drop('index',axis=1)
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,118,1.431053,-0.648101,-0.331664,-1.207383,-0.012786,0.704440,-0.732126,0.183139,-0.878334,...,0.112749,0.222296,-0.244591,-1.688158,0.609778,-0.088366,0.010438,-0.014499,20.0,0.0
1,406,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.0,1.0
2,472,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.0,1.0
3,1256,-2.594704,0.792116,-0.749193,-2.073082,-0.938225,-1.819945,-0.102909,1.103437,0.204788,...,0.173200,0.377772,0.148204,0.587936,0.071497,-0.153507,0.262471,-0.081060,1.0,0.0


In [51]:
fraud_count = df_balanced["Class"].value_counts()
fraud_rate = 100* fraud_count[1]/df.shape[0]
fraud_data = pd.concat([fraud_count, fraud_rate_series], axis=1).reset_index()
fraud_data.columns = ['Class', 'Count', 'Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,2,0.100705
1,1.0,2,NaN


In [55]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced.drop('Class', axis=1), df_balanced['Class'], test_size=0.2, random_state=101)
print(f'''X_train : {X_train.shape} X_test: {X_test.shape}y_train: {y_train.shape} y_test:{y_test.shape}''')

X_train : (3, 30) X_test: (1, 30)y_train: (3,) y_test:(1,)


In [56]:
randomForestModel = Pipeline({
    ('scaler',StandardScaler()),
    ('classifier',RandomForestClassifier())
})

randomForestModel.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [58]:
y_pred_logis = randomForestModel.predict(X_test)
y_pred_logis

array([0.])

In [59]:
cr=classification_report(y_test,y_pred_logis)
print(cr)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [67]:
with open(r'/content/drive/MyDrive/creditcard.pkl', 'wb') as fp:
  pickle.dump(randomForestModel, fp)